# Play Duration
**Date:** Monday, March 17, 2025

## Issues
- Some rows in `dt_playhistory` show incorrect values for `play_secs` and `play_time`


## Examples
- **To Do:** Show rows for March 14th, 2025

## Code

### Imports

In [69]:
import os
from datetime import datetime, timedelta

import sqlalchemy as sa
from sqlalchemy import create_engine, text

from environs import Env


### Environment

In [70]:
_basedir = os.path.abspath(os.curdir)
envfilename = os.path.join(_basedir, "notebooks", ".env")

env = Env(expand_vars=True)
env.read_env(envfilename)

True

### Connect to Database

In [71]:
db_url = env("SQLALCHEMY_DATABASE_URI")
schema = env("DB_SCHEMA")
engine = create_engine(db_url, echo=False)

### Database Helper Functions

**set_search_path()** - Sets the PostgreSQL search_path

In [72]:
@sa.event.listens_for(engine, "connect", insert=True)
def set_search_path(dbapi_connection, connection_record):
    """
    Set schema search path in database
    """
    sql = f"SET SESSION search_path TO {schema},public;"
    existing_autocommit = dbapi_connection.autocommit
    dbapi_connection.autocommit = True
    cursor = dbapi_connection.cursor()
    cursor.execute(sql)
    cursor.close()
    dbapi_connection.autocommit = existing_autocommit

## Sample Data
### Notes
- SQLAlchemy doesn't appear to support "`WHERE .. IN`" clauses in SELECT statements
- Workaround: Multiple `WHERE` clauses with `OR`

In [73]:
# Query #1 Find rows with large values in play_secs column
print("Query #1")

tbl = env("DB_DATA_TABLE", default="dt_playhistory")
columns = "id playdatetime play_secs play_time filename".split()
cols = ",".join(columns)
whr = " AND ".join([
	"playdate = :playdate",
	"play_secs > :play_secs"
])
params = {
	"playdate": "2025-03-14", 
	"play_secs": 1800,
}
sql = "\n".join([
	f"SELECT {cols}",
	f"FROM {tbl}",
	f"WHERE {whr};",
])

rowids = []
with engine.connect() as conn:
	for rowid, dt, secs, dur, fname in conn.execute(text(sql), params).fetchall():
		duration = timedelta(seconds=round(dur.total_seconds(), 0))
		stub = fname.lstrip("/home/patrick/")
		print(f"{rowid}) {dt.replace(microsecond=0).time()} | {duration} | {stub}")
		rowids.extend([rowid - 1, rowid, rowid + 1])


Query #1
343867) 10:45:19 | 1:02:58 | Music/Ultimate_80s/Disc_10/22 Starship - Nothings Gonna Stop Us.mp4
343891) 13:09:36 | 3:25:18 | Videos/TV/AllInTheFamily/Season_02/S02E20 Edith Gets a Mink.mp4
343934) 18:42:06 | 0:49:08 | Videos/Torrents/TV/LALaw/Season_06/S06E16 From Here to Paternity.mp4


In [75]:
# Query #2 - Find row before & after rows found in Query #1
print("Query #2")
params = {}
for i, rowid in enumerate(rowids):
	key = f"row{i+1}"
	params[key] = rowid

whr = " OR ".join([
	f"id = :{x}" for x in params
])

sql = "\n".join([
	f"SELECT {cols}",
	f"FROM {tbl}",
	f"WHERE {whr};",
])

with engine.connect() as conn:
	for rowid, dt, secs, dur, fname in conn.execute(text(sql), params).fetchall():
		duration = timedelta(seconds=round(dur.total_seconds(), 0))
		stub = fname.lstrip("/home/patrick/")
		print(f"{rowid}) {dt.replace(microsecond=0).time()} | {duration} | {stub}")


Query #2
343935) 04:05:36 | 0:02:52 | Music/Ultimate_80s/Disc_09/14 Men Without Hats - Safety Dance.mp4
343866) 10:41:06 | 0:04:13 | Music/Ultimate_80s/Disc_10/21 Poison - Mama's Fallen Angel.mp4
343867) 10:45:19 | 1:02:58 | Music/Ultimate_80s/Disc_10/22 Starship - Nothings Gonna Stop Us.mp4
343868) 11:48:17 | 0:04:59 | Music/Ultimate_80s/Disc_10/23 The Church - Under The Milky Way.mp4
343890) 13:08:03 | 0:01:33 | Videos/ydl/New_York_Post/20250312/Ontario_Premier_Doug_Ford_apologizes_to_Americans_amid_tariff_war_-_I_love_the_American_people-VP_6Njjb0HU-720.mkv
343891) 13:09:36 | 3:25:18 | Videos/TV/AllInTheFamily/Season_02/S02E20 Edith Gets a Mink.mp4
343892) 16:34:54 | 0:00:05 | Music/Ultimate_80s/Disc_09/01 Cyndi Lauper - True Colors.mp4
343933) 18:39:55 | 0:02:11 | Videos/ydl/CityNews/20250314/CityNews_bids_farewell_to_studios_at_Yonge_and_Dundas-yJQlmwZ-Dz8-720.mkv
343934) 18:42:06 | 0:49:08 | Videos/Torrents/TV/LALaw/Season_06/S06E16 From Here to Paternity.mp4
